Questa sezione di codice si occupa delle reti neurali. Grazie all'analisi di correlazione abbiamo capito quali statistiche sono correlate con i dati di Gol ed Assist. Daremo quindi in input alla rete neurale quelle statistiche e chiederemo in output, appunto, Gol ed Assist.

In [1]:
# Importo le librerie necessarie

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np 

In [2]:
# Nelle prossime celle andremo ad impostare la rete neurale, importando i file ed addestrandola con i file di input. 
# Andiamo a fare prima i gol e nella cella seguente la rete per gli assist

In [3]:
f1 = "attaccanti_19.xlsx"
f2 = "attaccanti_20.xlsx"
f3 = "attaccanti_21.xlsx"
f4 = "attaccanti_22.xlsx"
f5 = "attaccanti_23.xlsx"

# Elimino le colonne che abbiamo scoperto essere non utili per la nostra analisi

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK', 'PKatt', 
                        'CrdY', 'CrdR', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG','Cmp', 'Att','CrsPA', 'Cmp%', 'Tkl',
                        'TklW', 'Mid 3rd', 'Att 3rd', 'Mid 3rd.1','Succ%', 'TotDist', '2CrdY', 'Fld', 'Off','Crs', 'Won', 'Lost', 'Won%']

# Unisco i file Excel
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2).iloc[1:].reset_index(drop=True)
df3 = pd.read_excel(f3).iloc[1:].reset_index(drop=True)
df4 = pd.read_excel(f4).iloc[1:].reset_index(drop=True)
df_unito = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Rimuovo le colonne
df_unito = df_unito.drop(columns=colonne_da_eliminare, errors='ignore')

df5 = pd.read_excel(f5).iloc[1:].reset_index(drop=True)
df5 = df5.drop(columns=colonne_da_eliminare, errors='ignore')

# Separare le colonne in input e output
X = df_unito.iloc[:, 2:]  # Tutte le colonne eccetto le prime due
y = df_unito.iloc[:, :2]  # Le prime due colonne

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Costruzione della rete neurale
model = tf.keras.Sequential([tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(y_train.shape[1])])

# Compilazione
model.compile(optimizer='adam', loss='mean_squared_error')

# Addestramento
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Carica il nuovo file e pre-processa i dati
df_new = df5  

# Verifica che le colonne siano allineate con il DataFrame di addestramento
X_new = df_new[X.columns]

# Pre-elabora i dati nuovi
X_new_scaled = scaler.transform(X_new)

# Usa il modello per fare previsioni
y_pred = model.predict(X_new_scaled)

# Imposto pari a zero i valori negativi in quanto è impossibile realizzare un numero negativo di gol
 
y_pred = np.clip(y_pred, 0, None)  # Sostituisci i valori negativi con 0

# Visualizza le previsioni
df_new['Pred_Gls'] = y_pred[:, 0]  # Previsioni per la prima colonna (Gls)
df_new['Pred_Ast'] = y_pred[:, 1]  # Previsioni per la seconda colonna (Ast)

print(df_new[['Gls', 'Ast', 'Pred_Gls', 'Pred_Ast']])

# Calcola separatamente la somma delle previsioni per Gls e Ast
somma_goal_attesi = df_new['Pred_Gls'].sum()
somma_assist_attesi = df_new['Pred_Ast'].sum()

# Calcola separatamente la somma effettiva dei goal e assist nei dati originali
somma_goal = df_new['Gls'].sum()
somma_assist = df_new['Ast'].sum()

c:\Users\aless\miniconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 15.7977 - val_loss: 15.3490
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 11.3930 - val_loss: 9.4076
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.9470 - val_loss: 7.2573
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.9790 - val_loss: 5.7070
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.7566 - val_loss: 4.0641
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.1926 - val_loss: 3.2866
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.4754 - val_loss: 2.8839
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.1891 - val_loss: 2.7517
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.8950 - val_loss: 2.7563
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1249 - val_loss: 2.5009
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.9209 - val_loss: 2.5342
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss

In [4]:
# Risultati dei gol
print(f"La somma dei goal effettivi è {somma_goal}")
print(f"La somma dei goal ottenuti dalla rete neurale è {somma_goal_attesi}")

# Risultati degli assist

print(f"La somma degli assist effettivi è {somma_assist}")
print(f"La somma degli assist ottenuti dalla rete neurale è {somma_assist_attesi}")

# Errore

diff_gol = abs(somma_goal - somma_goal_attesi)
diff_ast = abs(somma_assist - somma_assist_attesi)

# Trovo l'errore medio dividendo per il numero di giocatori 

avg_err_gol = diff_gol/194  
avg_err_ast = diff_ast/194

print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_gol} Gol")
print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_ast} Assist")
print(f"L'errore medio per i gol è {avg_err_gol} per giocatore")
print(f"L'errore medio per gli assist è {avg_err_ast} per giocatore")

La somma dei goal effettivi è 622
La somma dei goal ottenuti dalla rete neurale è 632.2074584960938
La somma degli assist effettivi è 289
La somma degli assist ottenuti dalla rete neurale è 322.95050048828125
La differenza fra la somma effettiva e quella della rete neurale è di 10.20745849609375 Gol
La differenza fra la somma effettiva e quella della rete neurale è di 33.95050048828125 Assist
L'errore medio per i gol è 0.052615765443782214 per giocatore
L'errore medio per gli assist è 0.17500257983650128 per giocatore


Aumentando il numero delle epoche ovviamente la differenza fra la previsione della rete neurale e la somma effettiva delle statistiche richieste si assottiglia notevolmente. Il processo richiede più tempo ma restituisce risultati più precisi.